In [24]:
from datetime import datetime
from sklearn import metrics

import pickle
import numpy
import os
import sys

import time
from tqdm import tqdm

import Classes
from torch import optim, nn
from sklearn import utils
from Functions import *
from torch.autograd import Variable
from torch import cat, from_numpy

import warnings
warnings.filterwarnings("ignore") # to do remove

In [25]:
# ZEROUT_DUMMY_WORD = False
ZEROUT_DUMMY_WORD = True

## Load data
# mode = 'TRAIN-ALL'
mode = 'train'
#if len(sys.argv) > 1:
#    mode = sys.argv[1]
#    if not mode in ['TRAIN', 'TRAIN-ALL']:
#        print("ERROR! The two possible training settings are: ['TRAIN', 'TRAIN-ALL']")
#        sys.exit(1)

print("Running training in the {} setting".format(mode))

data_dir = mode

if mode in ['TRAIN-ALL']:
    q_train = numpy.load(os.path.join(data_dir, 'train-all.questions.npy'))
    a_train = numpy.load(os.path.join(data_dir, 'train-all.answers.npy'))
    q_overlap_train = numpy.load(os.path.join(data_dir, 'train-all.q_overlap_indices.npy'))
    a_overlap_train = numpy.load(os.path.join(data_dir, 'train-all.a_overlap_indices.npy'))
    y_train = numpy.load(os.path.join(data_dir, 'train-all.labels.npy'))
else:
    q_train = numpy.load(os.path.join(data_dir, 'train.questions.npy'))
    a_train = numpy.load(os.path.join(data_dir, 'train.answers.npy'))
    q_overlap_train = numpy.load(os.path.join(data_dir, 'train.q_overlap_indices.npy'))
    a_overlap_train = numpy.load(os.path.join(data_dir, 'train.a_overlap_indices.npy'))
    y_train = numpy.load(os.path.join(data_dir, 'train.labels.npy'))

q_dev = numpy.load(os.path.join(data_dir, 'dev.questions.npy'))
a_dev = numpy.load(os.path.join(data_dir, 'dev.answers.npy'))
q_overlap_dev = numpy.load(os.path.join(data_dir, 'dev.q_overlap_indices.npy'))
a_overlap_dev = numpy.load(os.path.join(data_dir, 'dev.a_overlap_indices.npy'))
y_dev = numpy.load(os.path.join(data_dir, 'dev.labels.npy'))
qids_dev = numpy.load(os.path.join(data_dir, 'dev.qids.npy'))

q_test = numpy.load(os.path.join(data_dir, 'test.questions.npy'))
a_test = numpy.load(os.path.join(data_dir, 'test.answers.npy'))
q_overlap_test = numpy.load(os.path.join(data_dir, 'test.q_overlap_indices.npy'))
a_overlap_test = numpy.load(os.path.join(data_dir, 'test.a_overlap_indices.npy'))
y_test = numpy.load(os.path.join(data_dir, 'test.labels.npy'))
qids_test = numpy.load(os.path.join(data_dir, 'test.qids.npy'))

Running training in the train setting


In [26]:
print('y_train', numpy.unique(y_train, return_counts=True))
print('y_dev', numpy.unique(y_dev, return_counts=True))
print('y_test', numpy.unique(y_test, return_counts=True))

print('q_train', q_train.shape)
print('q_dev', q_dev.shape)
print('q_test', q_test.shape)

print('a_train', a_train.shape)
print('a_dev', a_dev.shape)
print('a_test', a_test.shape)

y_train (array([0, 1], dtype=int32), array([4370,  348]))
y_dev (array([0, 1], dtype=int32), array([926, 222]))
y_test (array([0, 1], dtype=int32), array([1233,  284]))
q_train (4718, 33)
q_dev (1148, 33)
q_test (1517, 33)
a_train (4718, 40)
a_dev (1148, 40)
a_test (1517, 40)


In [27]:
numpy_rng = numpy.random.RandomState(123)
q_max_sent_size = q_train.shape[1]
a_max_sent_size = a_train.shape[1]
ndim = 5
dummy_word_id = numpy.max(a_overlap_train)
#print("Gaussian")
vocab_emb_overlap = numpy_rng.randn(dummy_word_id+1, ndim) * 0.25
vocab_emb_overlap[-1] = 0

In [28]:
fname = os.path.join(data_dir, 'emb_aquaint+wiki.txt.gz.ndim=50.bin.npy')
vocab_emb = numpy.load(fname)
ndim = vocab_emb.shape[1]
dummpy_word_idx = numpy.max(a_train)
print("Word embedding matrix size:", vocab_emb.shape)

Word embedding matrix size: (17023, 50)


In [12]:
n_outs = 2
n_epochs = 25
#n_epochs = 5
batch_size = 50
learning_rate = 0.1
max_norm = 0

In [29]:
print('batch_size', batch_size)
print('n_epochs', n_epochs)
print('learning_rate', learning_rate)
print('max_norm', max_norm)

batch_size 18
n_epochs 25
learning_rate 0.1
max_norm 0


In [30]:
## 1st conv layer.
# ndim = vocab_emb.shape[1] + vocab_emb_overlap.shape[1]
ndim = vocab_emb.shape[1]

print(vocab_emb.shape[1])
print(vocab_emb_overlap.shape[0])
print(vocab_emb_overlap.shape[1])
dropout_rate = 0.5
nkernels = 100
q_k_max = 1
a_k_max = 1

# filter_widths = [3,4,5]
q_filter_widths = [5]
a_filter_widths = [5]

num_input_channels = 1
input_shape = (batch_size, num_input_channels, q_max_sent_size + 2 * (max(q_filter_widths) - 1), ndim)
print(input_shape)

50
3
5
(18, 1, 41, 50)


In [32]:
model1 = Classes.SiameseNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model1.parameters(), lr=learning_rate)

In [33]:
#Train
train_loss = []
lookup = Classes.LookUpWord(vocab_emb)

#preparamos el dev set
dev_q_test = lookup.out_matrix(q_dev)
dev_a_test = lookup.out_matrix(a_dev)
best_dev = 0
best = False

for epoch in range(1, n_epochs+1):

    start_epoch = time.time()

    q_train, a_train, y_train = utils.shuffle(q_train, a_train, y_train)
    batch_id = 0
    for x_trainq, x_traina, y_train1 in zip(batch_gen(q_train, 50), batch_gen(a_train, 50), batch_gen(y_train, 50)):
        #model.h_t, model.c_t = repackage_hidden((model.h_t, model.c_t))
        model1.zero_grad() # reinicia el batch, para que no se quede con los gradientes del batch anterior
        batch_id += 1
        x_query =lookup.out_matrix(x_trainq)  # puede ser mas eficiente esto?

        x_answer = lookup.out_matrix(x_traina)
          #inputsq, inputsa, labels = Variable(x_query), Variable(x_answer), Variable(label)

        labels = Variable(from_numpy(numpy.array(y_train1)))
        output = model1(x_query, x_answer)
        batch_size= output.size(0)
        output = output.view(batch_size,2)

        labels = labels.long()
        loss = criterion(output, labels)
        train_loss.append(loss.data[0])

        optimizer.zero_grad()
          # del x_query?
        loss.backward()
        optimizer.step()
    end_epoch = time.time()
    took2 = end_epoch - start_epoch
    print('Train epoch: {} \tTook:{:.2f}'.format(epoch, took2))

    dev_pred = model1(dev_q_test, dev_a_test)
    normal_sc = score(dev_pred, y_dev)
    map_sc = map_score(qids_dev, dev_pred, y_dev)
    if map_sc > best_dev:
        best_dev = map_sc
        best = True

    if epoch % 5 == 0:
        # revisamos el mejor puntaje usando dev
        if not best:
            print("No ha habido mejora")
            break
        else:
            best = False



Train epoch: 1 	Took:2.87
Train epoch: 2 	Took:2.55
Train epoch: 3 	Took:2.39
Train epoch: 4 	Took:2.38
Train epoch: 5 	Took:2.36
Train epoch: 6 	Took:2.36
Train epoch: 7 	Took:2.49
Train epoch: 8 	Took:2.38
Train epoch: 9 	Took:2.35
Train epoch: 10 	Took:2.36
No ha habido mejora


In [37]:
#prediccion de ejemplo
trad = Classes.Traductor()

question = trad.traducir(q_test[57])
print("")
answer = trad.traducir(a_test[40])
wrong_answer = trad.traducir(a_test[4])

print(question)
print(answer)
print(wrong_answer)
print(y_test[1])
print(y_test[4])


 when   did   amtrak   begin   operations   ?                            
 amtrak   could   face   liquidation   if   it   fails   to   meet   the   deadline   .                             
 that   's   because   ms   .   palmer   is   a   witch   ,   the   high   priestess   of   a   group   that   practices   wicca   at   fort   hood   with   the   knowledge   and   approval   of   the   u.s   .   army   .        
1
0


In [21]:
query_t = lookup.out_matrix(q_test)
answer_t = lookup.out_matrix(a_test)

y_pred = model1(query_t, answer_t)
print(y_pred[1])
print(y_pred[4])

print(score(y_pred, y_test))
print(map_score(qids_test, y_pred, y_test))
print(y_pred.sum())

Variable containing:
1.00000e-04 *
  7.4296  6.6514
[torch.FloatTensor of size 1x2]

Variable containing:
1.00000e-04 *
  8.0515  6.4145
[torch.FloatTensor of size 1x2]

47.13249835201055
0.6030210164570313
Variable containing:
 2
[torch.FloatTensor of size 1]

